# 第二课 词向量

第二课学习目标
- 学习词向量的概念
- 用Skip-thought模型训练词向量
- 学习使用PyTorch dataset和dataloader
- 学习定义PyTorch模型
- 学习torch.nn中常见的Module
    - Embedding
- 学习常见的PyTorch operations
    - bmm
    - logsigmoid
- 保存和读取PyTorch模型
    

第二课使用的训练数据可以从以下链接下载到。

链接:https://pan.baidu.com/s/1tFeK3mXuVXEy3EMarfeWvg  密码:v2z5

在这一份notebook中，我们会（尽可能）尝试复现论文[Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)中训练词向量的方法. 我们会实现Skip-gram模型，并且使用论文中noice contrastive sampling的目标函数。

这篇论文有很多模型实现的细节，这些细节对于词向量的好坏至关重要。我们虽然无法完全复现论文中的实验结果，主要是由于计算资源等各种细节原因，但是我们还是可以大致展示如何训练词向量。

以下是一些我们没有实现的细节
- subsampling：参考论文section 2.3

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from torch.nn.parameter import Parameter

from collections import Counter
import numpy as np
import random
import math

import pandas as pd
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

USE_CUDA = torch.cuda.is_available()

# 为了保证实验结果可以复现，我们经常会把各种random seed固定在某一个值
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)
    
# 设定一些超参数
    
K = 100 # number of negative samples
C = 3 # nearby words threshold
NUM_EPOCHS = 2 # The number of epochs of training
MAX_VOCAB_SIZE = 30000 # the vocabulary size
BATCH_SIZE = 128 # the batch size
LEARNING_RATE = 0.2 # the initial learning rate
EMBEDDING_SIZE = 100
       
    
LOG_FILE = "word-embedding.log"

# tokenize函数，把一篇文本转化成一个个单词
def word_tokenize(text):
    return text.split()

**两个模块的区别：**[torch.nn 和 torch.functional 的区别](https://blog.csdn.net/hawkcici160/article/details/80140059)

30000

- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [2]:
with open("data/text8.train.txt", "r") as fin:
    text = fin.read()
    
text = [w for w in word_tokenize(text.lower())]
vocab = dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))
vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))
idx_to_word = [word for word in vocab.keys()] 
word_to_idx = {word:i for i, word in enumerate(idx_to_word)}

word_counts = np.array([count for count in vocab.values()], dtype=np.float32)
word_freqs = word_counts / np.sum(word_counts)
word_freqs = word_freqs ** (3./4.)
word_freqs = word_counts / np.sum(word_freqs) # 用来做 negative sampling
VOCAB_SIZE = len(idx_to_word)
VOCAB_SIZE

30000

### 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字，然后用subsampling预处理这些文字。
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

30000

In [9]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        ''' text: a list of words, all text from the training dataset
            word_to_idx: the dictionary from word to idx
            idx_to_word: idx to word mapping
            word_freq: the frequency of each word
            word_counts: the word counts
        '''
        super(WordEmbeddingDataset, self).__init__()
        self.text_encoded = [word_to_idx.get(t, VOCAB_SIZE-1) for t in text]
        self.text_encoded = torch.Tensor(self.text_encoded).long()
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.word_freqs = torch.Tensor(word_freqs)
        self.word_counts = torch.Tensor(word_counts)
        
    def __len__(self):
        ''' 返回整个数据集（所有单词）的长度
        '''
        return len(self.text_encoded)
        
    def __getitem__(self, idx):
        ''' 这个function返回以下数据用于训练
            - 中心词
            - 这个单词附近的(positive)单词
            - 随机采样的K个单词作为negative sample
        '''
        center_word = self.text_encoded[idx]
        pos_indices = list(range(idx-C, idx)) + list(range(idx+1, idx+C+1))
        pos_indices = [i%len(self.text_encoded) for i in pos_indices]
        pos_words = self.text_encoded[pos_indices] 
        neg_words = torch.multinomial(self.word_freqs, K * pos_words.shape[0], True)
        
        return center_word, pos_words, neg_words 

In [4]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        super(WordEmbeddingDataset, self).__init__()
        self.text_encoded = [word_to_idx.get(word, word_to_idx["<unk>"]) for word in text]
        #把每个单词出现的次数赋给text_encoded
        #word_to_idx.get——dict.get方法，取出相关数据的对应，例子在https://www.runoob.com/python/att-dictionary-get.html
        #[a1 for a2 in b]的运行逻辑为 先在b中取出数据放在a2，然后再把数据a2传给a1，a1在起赋值作用，之后开始下一轮循环，再从b中取下一个数据给a2
        #类如[len(a1) for a2 in b] 在b中取出第一个数据给a2，然后把a2数据给a1，再用a1来计算函数len，得出的结果用于赋值，之后开始下一轮循环，从b中取下一个数据给a2
        self.text_encoded = torch.LongTensor(self.text_encoded)
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.word_freqs = torch.Tensor(word_freqs)
        self.word_counts = torch.Tensor(word_counts)
                                             
    def __len__(self):
        #数据集有多少item
        return len(self.text_encoded)
    def __getitem__(self, idx):
        center_word = self.text_encoded(idx)
        pos_indices = list(range(idx-C)) + list(range(idx+1, idx+C+1))
        pos_indices = [i % len(self.text_encoded) for i in pos_indices]
        pos_words = self.text_encoded[pos_indices]
        neg_words = torch.multinomial(self.word_freqs, K * pos_words.shape[0], True)
        
        return center_word, pos_words, neg_words 

创建dataset和dataloader

In [14]:
dataset = WordEmbeddingDataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)
dataloader = tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)    

In [24]:
for i, (center_word, pos_words, neg_words) in enumerate(dataloader):
    print(center_word, pos_words, neg_words)
    if i > 5:
        break

tensor([ 4813,  3139,    11,     5,   194,     1,  3015,    46,    59,   155,
          127,   741,   461, 10485,   133,     0, 25752,     1,     0,   108,
          833,     2,     0, 16267, 29999,     1,     0,   152,   833,  3493,
            0,   194,    10,   186,    59,     4,     5, 10620,   213,     6,
         1332,   102,   437,    19,    59,  2764,   355,     6,  3625,     0,
          709,     1,   364,    26,    40,    37,    53,   527,    97,    11,
            5,  1398,  2929,    18,   562,   691,  6644,     0,   252,  4813,
           10,  1043,    27,     0,   316,   247, 29999,  2964,   789,   189,
         4813,    11,     5,   201,   569,    10,     0,  1107,    19,  2581,
           25,  8819,     2,   273,    31,  4089,   140,    58,    25,  6494,
         4195,     1,   153,    32,   355,  4813,    37,  1128,     6,   441,
          344,  1794,    19,  4682,     0,  6645,     1,  7542,  1756,   567,
            0,    94,     0,   252, 11304,    11,    52,  6644])

        [ 5310,    23,     0, 12153,   889,    10]]) tensor([[ 9004,  3510,     2,  ...,     3,     6,    58],
        [    4,  6162,  1393,  ...,   214,   390,   177],
        [   35,   126,    23,  ..., 29999,     3,  3158],
        ...,
        [ 4673, 29999,  1072,  ...,     2, 29999,  6263],
        [21063,     5,   191,  ...,  4814,    13,   388],
        [ 2717,    13,  5472,  ...,     2,     6,   187]])
tensor([12153,   889,    10,  6718,     4,    32,   137,    35,  1356,     0,
         2908,     1, 14602,   889, 29999,   154,    99,  3502,    38,   652,
          445,     6,    89,     2,  3015,    43,   889,    11,    45,  3569,
           54,    11, 15904,  1529,    13,  2896,     4,    44,   259, 13571,
         1165,   153,    35,    88,  3153,  1104,    51,    38,   762,   445,
            6,    89,  1034,   524,     2,  1787,     4,  5687,    23,  1533,
            1,  5057,     2,  1303, 13571,    14,  2582,     1, 11304,    33,
           35,    88, 27615, 29999,   9

torch.utils.data.DataLoader理解：https://blog.csdn.net/qq_36653505/article/details/83351808

### 定义PyTorch模型

In [99]:
class EmbeddingModel(nn.Module):
    def __init__(self, vcab_size, embed_size):
        super(EmbeddingModel, self).__init__()
        
        self.vocab_size = vcab_size
        self.embed_size = embed_size
        
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size)
        self.out_embed = nn.Embedding(self.vocab_size, self.embed_size)
        
    def forward(self, input_labels, pos_labels, neg_labels):
        # input_label: [batch_size]
        #pos_labels: [batch_size,(window_size * z)]
        #neg_labels: [batch_size, (window_size * z * K)]
        
        input_embedding = self.in_embed(input_labels) #batch_size * embed_size
        pos_embedding = self.in_embed(pos_labels)
        neg_embedding = self.in_embed(neg_labels)
        
        input_embedding = input_embedding.unsqueeze(2)
        pos_dot = torch.bmm(pos_embedding, input_embedding).squeeze(2)
        neg_dot = torch.bmm(pos_embedding, -input_embedding).squeeze(2)
        log_pos = F.logsigmoid(pos_dot).sum(1)
        log_neg = F.logsigmoid(neg_dot).sum(1)
        
        loss = log_pos + log_neg
        return -loss
    def input_embedding(self):
        return self.in_embed.weight.cpu().numpy()

定义一个模型以及把模型移动到GPU

In [100]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()

下面是评估模型的代码，以及训练模型的代码

In [101]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
for e in range(NUM_EPOCHS):
    for i, (input_labels, pos_labels, neg_labels) in enumerate (dataloader):
#         print(input_labels, pos_labels, neg_lagels)
#         if i > 5:
#             break
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()
            
        optimizer.zero_grad()
        loss = model(input_labels, pos_labels, neg_lagels).mean()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
                print("epoch", e, "iteration", i, loss.item())
                

epoch 0 iteration 0 60.08375549316406
epoch 0 iteration 100 35.99202346801758
epoch 0 iteration 200 27.963699340820312
epoch 0 iteration 300 26.30188751220703
epoch 0 iteration 400 21.880258560180664
epoch 0 iteration 500 24.499858856201172
epoch 0 iteration 600 23.51007652282715
epoch 0 iteration 700 24.086776733398438


KeyboardInterrupt: 

训练模型：
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，以及在验证数据集上做模型的评估

## 在 MEN 和 Simplex-999 数据集上做评估

## 寻找nearest neighbors

## 单词之间的关系